In [5]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from catboost import CatBoostRegressor, Pool

train_data = pd.read_csv('C:/Users/HUAWEI/ML/task_4/train.csv')
test_data = pd.read_csv('C:/Users/HUAWEI/ML/task_4/test.csv')
train_data = train_data.drop(columns=["Id"])
val_ids = test_data["Id"]
test_data = test_data.drop(columns=["Id"])

for feature in train_data.columns:
    percent = train_data[feature].isnull().sum() /  train_data.shape[0]
    if (percent > 0.7):
        train_data = train_data.drop(columns = feature)
        test_data = test_data.drop(columns = feature)

cat_features = []
iter = 0
for feature in train_data.columns[:-1]:
    if (train_data[feature].dtype == 'object'): cat_features.append(iter)
    iter +=1

X_train, X_test, y_train, y_test = train_test_split(train_data.drop(columns='SalePrice').values, np.log1p(train_data['SalePrice'].values), test_size = 0.2, random_state = 98987)
X_train = SimpleImputer(strategy='most_frequent').fit_transform(X_train)
X_test = SimpleImputer(strategy='most_frequent').fit_transform(X_test)
test_data = SimpleImputer(strategy='most_frequent').fit_transform(test_data)

In [3]:
catr = CatBoostRegressor(cat_features=cat_features)
train_pool = Pool(X_train, y_train, cat_features=cat_features)
dev_pool = Pool(X_test, y_test, cat_features=cat_features)
catr.fit(train_pool, eval_set = dev_pool, verbose = 0, plot = False)

print('Error on train: ', metrics.mean_absolute_error(catr.predict(X_train), y_train))
print('Error on test: ', metrics.mean_absolute_error(catr.predict(X_test), y_test))

Error on train:  0.038426950347731775
Error on test:  0.08808372671727617


In [6]:
df = pd.DataFrame({'Id':val_ids.values, 'SalePrice':np.expm1(catr.predict(test_data))})
df.to_csv('catBoost.csv',index=False)